In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from utils.preprocessing import preprocess_df
import pickle
from graphviz import Source
from sklearn.tree import export_graphviz
import os

from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [4]:
#### Select dataset ####
all_datasets = ['adult', 'german', 'compas', 'diabetes', 'breast_cancer']

In [35]:
def generate_tree_info(dataset_name: str):
    if dataset_name == 'adult':
        dataset_loading_fn = load_adult_df
    elif dataset_name == 'german':
        dataset_loading_fn = load_german_df
    elif dataset_name == 'compas':
        dataset_loading_fn = load_compas_df
    elif dataset_name == 'diabetes':
        dataset_loading_fn = load_diabetes_df
    elif dataset_name == 'breast_cancer':
        dataset_loading_fn = load_breast_cancer_df
    else:
        raise Exception("Unsupported dataset")

    df_info = preprocess_df(dataset_loading_fn)

    path='./saved_models'
    destination_folder = './tree_info'
    storing_folder= f'{path}/{dataset_name}'

    dt = pickle.load(open(f'{storing_folder}/dt.p', 'rb'))
    rfc = pickle.load(open(f'{storing_folder}/rfc.p', 'rb'))

    features_importance_df = pd.DataFrame([], columns= ['Model'] + df_info.ohe_feature_names)
    features_importance_df.loc[0, df_info.ohe_feature_names] = dt.feature_importances_
    features_importance_df.loc[0, 'Model'] = 'DecisionTree'
    features_importance_df.loc[1, df_info.ohe_feature_names] = rfc.feature_importances_
    features_importance_df.loc[1, 'Model'] = 'RandomForest'

    features_importance_df.to_csv(os.path.join(destination_folder, f'{dataset_name}_feature_importancde.csv'))

    dot_data = export_graphviz(dt, out_file=None, feature_names=df_info.ohe_feature_names, class_names=df_info.possible_outcomes, rounded=True, proportion=False, precision=2, filled=True)
    graph = Source(dot_data)
    graph.format = 'svg'
    graph.render(f'./{destination_folder}/{dataset_name}_tree',view=True)


In [36]:
for d in all_datasets:
    generate_tree_info(d)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.210464 to fit


FileNotFoundError: [Errno 2] No such file or directory: './saved_models/german/dt.p'